In [98]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime as dt
import csv
from dateutil.parser import parse
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 200)

path = 'N:\\Operations Intelligence\\Operations Research\\Merchandising vs Operations\\*.csv'
files = glob.glob(path)

test_file = files[2]
print(test_file)
mfst = pd.read_csv(test_file, usecols=np.arange(0,25), names=["C"+str(i) for i in np.arange(1,26)])

## Exctract date from first column
rte_date = re.search(r'[0-9]{8}', str(test_file))
rte_date = dt.strptime(rte_date.group(), '%m%d%Y').date()
print(rte_date)

mfst['Date'] = rte_date
if re.search("STL", test_file).group(0) == 'STL':
    mfst['Warehouse'] = 'STL'
elif re.search("KC", test_file).group(0) == 'KC':
    mfst['Warehouse'] = 'KC'
elif re.search("COL", test_file).group(0) == 'COL':
    mfst['Warehouse'] = 'COL'
else:
    mfst['Warehouse'] = 'SPFD'
    
    
mfst.head()

N:\Operations Intelligence\Operations Research\Merchandising vs Operations\STL 05042017 Driver Manifest All Routes.csv
2017-05-04


C1   C2   C3   C4   C5   C6   C7   C8  C9  C10  C11  C12  C13  C14  C15  C16  C17  C18  C19  C20  C21  C22            C23  C24  C25        Date Warehouse
0                                   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN            NaN  NaN  NaN  2017-05-04       STL
1            Roadnet Technologies, Inc.  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN            NaN  NaN  NaN  2017-05-04       STL
2                                   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN            NaN  NaN  NaN  2017-05-04       STL
3  Date / Time Printed:  5/8/2017 10:28  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  Page  1 of  5  NaN  NaN  2017-05-04       STL
4                                   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN            NaN  NaN  NaN  2017-05-04       STL

In [99]:
## Extract RTE IDs
raw_rtes = todays_rtes = mfst.loc[mfst.C1.astype(str).str.contains('Route Id: '), 'C1']
raw_rtes = pd.DataFrame({'RouteId':raw_rtes}).reset_index(drop=False)

## String manipulations
todays_rtes = [rte.replace('Route Id: ','') for rte in todays_rtes]
todays_rtes = raw_rtes.RouteId = [rte.replace(' ', '') for rte in todays_rtes]
todays_rtes = pd.unique(todays_rtes)

print('Raw route values : \n', raw_rtes.head(10))
print('Processed route values: \n', todays_rtes)

Raw route values : 
    index               RouteId
0      8  R00015-CUBA/SULLIVAN
1     85  R00015-CUBA/SULLIVAN
2    176  R00015-CUBA/SULLIVAN
3    267  R00015-CUBA/SULLIVAN
4    358  R00015-CUBA/SULLIVAN
5    410    R00018-ROLLA/SALEM
6    487    R00018-ROLLA/SALEM
7    578    R00018-ROLLA/SALEM
8    690           R00021-TROY
9    767           R00021-TROY
Processed route values: 
 ['R00015-CUBA/SULLIVAN' 'R00018-ROLLA/SALEM' 'R00021-TROY' 'R00023-POTOSI'
 'R00030-CRESTWOOD/FENTON' 'R00031-SOUTHCOUNTY' 'R00032-FERGUSON'
 'R00033-ARNOLD' 'R00034-OFALLON' 'R00035-WENTZVILLE/NEWMELLE'
 'R00036-STPETERS' 'R00037-CHESTERFIELD' 'R00038-SAINTCHARLES'
 'R00039-MANCHESTER' 'R00040-SOULARD' 'R00041-KIRKWOOD' 'R00043-SOUTHCITY'
 'R00045-EUREKA/PACIFIC' 'R00051-ROCKROAD' 'R00053-FLORISSANT'
 'R00054-UCITY' 'R00060-CLAYTON' 'R00061-DOWNTOWN' 'R00063-CWE'
 'R00410-HAYTI' 'R00415-PIEDMONT' 'R00419-DEXTER' 'X00071-MANCHESTER/NORTH']


In [100]:
## Get start and end of route ID by using index from above
minmax = pd.DataFrame(raw_rtes.groupby('RouteId')['index'].agg({'RouteId':{'min':np.min, 'max':np.max}}))
minmax.columns = ['%s%s' % (a, '|%s' % b if b else '') for a, b in minmax.columns]

df_temp = pd.DataFrame()
for i, mm_row in minmax.iterrows():
    rte_id = str(mm_row.name)
    min_ix = int(mm_row[0])
    max_ix = int(mm_row[1])
    new_rows = {min_ix: rte_id, max_ix: rte_id}
    df = pd.DataFrame.from_dict(new_rows, orient='index')
    df.rename(columns={0:'RouteId'}, inplace=True)
    df_temp = df_temp.append(df)
    
new_ix = pd.Index(np.arange(np.min(df_temp.index.values), np.max(df_temp.index.values)))
df_temp = df_temp.reindex(new_ix)
df_temp.RouteId.fillna(method='ffill', inplace=True)
print(df_temp.head(), '\n\n', df_temp.tail())


mfst = mfst.join(df_temp)
mfst.RouteId.fillna(method='ffill', inplace=True)
# #mfst.RouteId.fillna(method='bfill', inplace=True)

expected_rtes = pd.unique(mfst['RouteId'])
# # print(expected_rtes)
print(expected_rtes)

                 RouteId
8   R00015-CUBA/SULLIVAN
9   R00015-CUBA/SULLIVAN
10  R00015-CUBA/SULLIVAN
11  R00015-CUBA/SULLIVAN
12  R00015-CUBA/SULLIVAN 

                        RouteId
10592  X00071-MANCHESTER/NORTH
10593  X00071-MANCHESTER/NORTH
10594  X00071-MANCHESTER/NORTH
10595  X00071-MANCHESTER/NORTH
10596  X00071-MANCHESTER/NORTH
[nan 'R00015-CUBA/SULLIVAN' 'R00018-ROLLA/SALEM' 'R00021-TROY'
 'R00023-POTOSI' 'R00030-CRESTWOOD/FENTON' 'R00031-SOUTHCOUNTY'
 'R00032-FERGUSON' 'R00033-ARNOLD' 'R00034-OFALLON'
 'R00035-WENTZVILLE/NEWMELLE' 'R00036-STPETERS' 'R00037-CHESTERFIELD'
 'R00038-SAINTCHARLES' 'R00039-MANCHESTER' 'R00040-SOULARD'
 'R00041-KIRKWOOD' 'R00043-SOUTHCITY' 'R00045-EUREKA/PACIFIC'
 'R00051-ROCKROAD' 'R00053-FLORISSANT' 'R00054-UCITY' 'R00060-CLAYTON'
 'R00061-DOWNTOWN' 'R00063-CWE' 'R00410-HAYTI' 'R00415-PIEDMONT'
 'R00419-DEXTER' 'X00071-MANCHESTER/NORTH']


In [101]:
## for future, SPFD has 3 cust who have colons
spfd_colon_custs = [5007745,5007746,5007747]

## Extract Customers -- maintain index
raw_cust = mfst.loc[~mfst.C5.astype(str).str.contains(':'), 'C5']
raw_cust = pd.DataFrame({'Customer':raw_cust}).reset_index(drop=False)

## String manipulations -- drop index for values
customers = mfst.loc[~mfst.C5.astype(str).str.contains(':'), 'C5']
customers = [c for c in customers if 'Service Windows' not in str(c) and 'na' not in str(c) 
             and 'Location Name' not in str(c) and 'Odometer Out:' not in str(c)]
customers = pd.unique(customers)

## Get start and end of route ID by using index from above
minmax = pd.DataFrame(raw_cust.groupby('Customer')['index'].agg({'Customer':{'min':np.min, 'max':np.max}}))
minmax.columns = ['%s%s' % (a, '|%s' % b if b else '') for a, b in minmax.columns]

print(minmax.head())
def make_windows(winz):
    if ',' not in str(winz):
        try:
            w1 = str(winz).split('-')[0]
            w2 = str(winz).split('-')[1]
            w3 = np.nan
            w4 = np.nan
        except IndexError:
            w1 = w2 = w3 = w4 = np.nan
    else:
        try:
            w1 = str(winz).split('-')[0]
            w2 = str(winz).split('-')[1]
            new_winz = str(winz).split(',') 
            new_winz = new_winz[1]
            w3 = str(winz).split('-')[0]
            w4 = str(winz).split('-')[1]
        except IndexError:
            w1 = w2 = w3 = w4 = np.nan
            
    return w1, w2, w3, w4

new_df = pd.DataFrame()
for i, mm_row in minmax.iterrows():
    cust_name = mm_row.name
    IX = mm_row[1]
    winz = mfst.loc[IX+4, 'C5']
    
    w1, w2, w3, w4 = make_windows(winz)   
    new_row = {IX: {'Customer':cust_name, 'CustomerId': mfst.loc[IX, 'C3'],
                   'Stop': mfst.loc[IX, 'C2'], 'Cases': mfst.loc[IX, 'C22'],
                   'Bottles':  mfst.loc[IX, 'C25'], 'ServiceWindows': mfst.loc[IX+4, 'C5'],
                   'BeginWindow1':w1, 'EndWindow1':w2, 'BeginWindow2':w3, 'EndWindow2':w4
                   }}
    df = pd.DataFrame.from_dict(new_row, orient='index')
    new_df = new_df.append(df)

print(new_df.head())

mfst = mfst.join(new_df)
mfst.head(30)
mfst[['Customer']].fillna(method='ffill', inplace=True)

cols_for_output = ['Warehouse','Date','RouteId','Customer','CustomerId','Stop','Cases','Bottles',
                   'ServiceWindows','BeginWindow1','EndWindow1','BeginWindow2','EndWindow2']

mfst[cols_for_output].head()

                          Customer|max  Customer|min
Customer                                            
109 BREWHOUSE                     3429          3429
2720 CHEROKEE                     5948          5948
5 ONE 5 PIT STOP                  9763          9763
5 STAR LIQUOR AND MORE            7858          7858
7 ELEVEN #19501 ST LOUIS          8970          8970
     BeginWindow1 Stop Bottles ServiceWindows EndWindow1                  Customer EndWindow2  Cases CustomerId BeginWindow2
3429        10:00   30    0.00    10:00-17:00      17:00             109 BREWHOUSE        NaN   5.00      13108          NaN
5948        09:00   32    0.00    09:00-16:00      16:00             2720 CHEROKEE        NaN   6.00      13421          NaN
9763        07:00    9    2.00    07:00-19:30      19:30          5 ONE 5 PIT STOP        NaN   4.00    4002342          NaN
7858        09:00   24   15.00    09:00-18:00      18:00    5 STAR LIQUOR AND MORE        NaN  22.00      13554          NaN
8970

C:\Users\pmwash\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


Warehouse        Date RouteId Customer CustomerId Stop Cases Bottles ServiceWindows BeginWindow1 EndWindow1 BeginWindow2 EndWindow2
0       STL  2017-05-04     NaN      NaN        NaN  NaN   NaN     NaN            NaN          NaN        NaN          NaN        NaN
1       STL  2017-05-04     NaN      NaN        NaN  NaN   NaN     NaN            NaN          NaN        NaN          NaN        NaN
2       STL  2017-05-04     NaN      NaN        NaN  NaN   NaN     NaN            NaN          NaN        NaN          NaN        NaN
3       STL  2017-05-04     NaN      NaN        NaN  NaN   NaN     NaN            NaN          NaN        NaN          NaN        NaN
4       STL  2017-05-04     NaN      NaN        NaN  NaN   NaN     NaN            NaN          NaN        NaN          NaN        NaN

In [102]:
## Filter out some nonsense
ISNAN = mfst['Stop'].isnull()
mfst = mfst.loc[ISNAN == False, cols_for_output]
BADVALS = ['Location Name']
mfst = mfst[~mfst.Customer.isin(BADVALS)]

## Reste index for slicing
mfst.set_index(keys=['Date','Warehouse','RouteId','CustomerId'], inplace=True, drop=False)
mfst.head()

Warehouse        Date               RouteId                   Customer CustomerId Stop  Cases Bottles ServiceWindows BeginWindow1 EndWindow1  \
Date       Warehouse RouteId              CustomerId                                                                                                                                                
2017-05-04 STL       R00015-CUBA/SULLIVAN 30194            STL  2017-05-04  R00015-CUBA/SULLIVAN  WALMART #0065 SULLIVAN SC      30194    1  48.00    0.00    05:00-11:00        05:00      11:00   
                                          30019            STL  2017-05-04  R00015-CUBA/SULLIVAN           DMG PETROLEUM CO      30019    2   4.00    0.00    06:00-13:00        06:00      13:00   
                                          30018            STL  2017-05-04  R00015-CUBA/SULLIVAN           MACE SUPERMARKET      30018    3  62.00    0.00    06:30-14:00        06:30      14:00   
                                          30480            STL  2017-05-04  R00015-CUBA/SULLIVAN              WAREHOUSE THE      30480    4  14.00   48.00    07:00-13:00        07:00      13:00   
                                          30475            STL  2017-05-04  R00015-CUBA/SULLIVAN         WALMART #0895 CUBA      30475    5   9.00    0.00    07:00-12:00        07:00      12:00   

                                                     BeginWindow2 EndWindow2  
Date       Warehouse RouteId              CustomerId                          
2017-05-04 STL       R00015-CUBA/SULLIVAN 30194               NaN        NaN  
                                          30019               NaN        NaN  
                                          30018               NaN        NaN  
                                          30480               NaN        NaN  
                                          30475               NaN        NaN

In [103]:
## Create datetime objects 
def make_datetime(rte_date, date_vector):
    for dat in date_vector:
        if dat != 'nan':
            try:
                DAT = dt.strptime(str(str(rte_date) + ' ' + str(dat)), '%Y-%m-%d %H:%M')
            except ValueError:
                DAT = pd.NaT
    return DAT

## Format as Datetime for operations
mfst.BeginWindow1 = make_datetime(rte_date, mfst.BeginWindow1)
mfst.EndWindow1 = make_datetime(rte_date, mfst.EndWindow1)
mfst.BeginWindow2 = make_datetime(rte_date, mfst.BeginWindow2)
mfst.EndWindow2 = make_datetime(rte_date, mfst.EndWindow2)

## Get N hours available in AM and PM
mfst['HoursAvailableWin1'] = mfst.EndWindow1 - mfst.BeginWindow1
mfst['HoursAvailableWin2'] = mfst.EndWindow2 - mfst.BeginWindow2

mfst.head()

Warehouse        Date               RouteId                   Customer CustomerId Stop  Cases Bottles ServiceWindows        BeginWindow1  \
Date       Warehouse RouteId              CustomerId                                                                                                                                            
2017-05-04 STL       R00015-CUBA/SULLIVAN 30194            STL  2017-05-04  R00015-CUBA/SULLIVAN  WALMART #0065 SULLIVAN SC      30194    1  48.00    0.00    05:00-11:00 2017-05-04 10:00:00   
                                          30019            STL  2017-05-04  R00015-CUBA/SULLIVAN           DMG PETROLEUM CO      30019    2   4.00    0.00    06:00-13:00 2017-05-04 10:00:00   
                                          30018            STL  2017-05-04  R00015-CUBA/SULLIVAN           MACE SUPERMARKET      30018    3  62.00    0.00    06:30-14:00 2017-05-04 10:00:00   
                                          30480            STL  2017-05-04  R00015-CUBA/SULLIVAN              WAREHOUSE THE      30480    4  14.00   48.00    07:00-13:00 2017-05-04 10:00:00   
                                          30475            STL  2017-05-04  R00015-CUBA/SULLIVAN         WALMART #0895 CUBA      30475    5   9.00    0.00    07:00-12:00 2017-05-04 10:00:00   

                                                              EndWindow1 BeginWindow2 EndWindow2  HoursAvailableWin1  HoursAvailableWin2  
Date       Warehouse RouteId              CustomerId                                                                                      
2017-05-04 STL       R00015-CUBA/SULLIVAN 30194      2017-05-04 17:00:00          NaT        NaT            07:00:00                 NaT  
                                          30019      2017-05-04 17:00:00          NaT        NaT            07:00:00                 NaT  
                                          30018      2017-05-04 17:00:00          NaT        NaT            07:00:00                 NaT  
                                          30480      2017-05-04 17:00:00          NaT        NaT            07:00:00                 NaT  
                                          30475      2017-05-04 17:00:00          NaT        NaT            07:00:00                 NaT

In [105]:
## Make duration into a floating point
mfst['HoursAvailableWin1'] = [np.float64(s.split(':')[0].split('days ')[1]) for s in mfst['HoursAvailableWin1'].astype(str)] 

print(mfst.head())

pd.unique(mfst.RouteId).tolist() in expected_rtes

                                                     Warehouse        Date               RouteId                   Customer CustomerId Stop  Cases Bottles ServiceWindows        BeginWindow1  \
Date       Warehouse RouteId              CustomerId                                                                                                                                            
2017-05-04 STL       R00015-CUBA/SULLIVAN 30194            STL  2017-05-04  R00015-CUBA/SULLIVAN  WALMART #0065 SULLIVAN SC      30194    1  48.00    0.00    05:00-11:00 2017-05-04 10:00:00   
                                          30019            STL  2017-05-04  R00015-CUBA/SULLIVAN           DMG PETROLEUM CO      30019    2   4.00    0.00    06:00-13:00 2017-05-04 10:00:00   
                                          30018            STL  2017-05-04  R00015-CUBA/SULLIVAN           MACE SUPERMARKET      30018    3  62.00    0.00    06:30-14:00 2017-05-04 10:00:00   
                                   

C:\Users\pmwash\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


False

In [106]:
mfst.groupby(['RouteId'])['HoursAvailableWin1'].mean()

RouteId
R00015-CUBA/SULLIVAN          7.0
R00018-ROLLA/SALEM            7.0
R00021-TROY                   7.0
R00023-POTOSI                 7.0
R00030-CRESTWOOD/FENTON       7.0
R00031-SOUTHCOUNTY            7.0
R00032-FERGUSON               7.0
R00033-ARNOLD                 7.0
R00034-OFALLON                7.0
R00035-WENTZVILLE/NEWMELLE    7.0
R00036-STPETERS               7.0
R00037-CHESTERFIELD           7.0
R00038-SAINTCHARLES           7.0
R00039-MANCHESTER             7.0
R00040-SOULARD                7.0
R00041-KIRKWOOD               7.0
R00043-SOUTHCITY              7.0
R00045-EUREKA/PACIFIC         7.0
R00051-ROCKROAD               7.0
R00053-FLORISSANT             7.0
R00054-UCITY                  7.0
R00060-CLAYTON                7.0
R00061-DOWNTOWN               7.0
R00063-CWE                    7.0
R00410-HAYTI                  7.0
R00415-PIEDMONT               7.0
R00419-DEXTER                 7.0
X00071-MANCHESTER/NORTH       7.0
Name: HoursAvailableWin1, dtype: float64